In [22]:
import pandas as pd

Ts = [0.25, 0.5, 0.75, 1.0, 1.25, 1.5]
# TEST
T = Ts[5]      

R = 5
P = 10

file_prefix = 'results/VQA_res/GPT35_JSON/'

tmp_df = pd.read_json(f"{file_prefix}analyt_df_T0.5.json")
print(tmp_df.columns)
tmp_df = pd.read_json(f"{file_prefix}paraphrased_questions_T0.5.json")
print(tmp_df.columns)
del tmp_df

Index(['Reference Question', 'Generated Question', 'P_0', 'P_1', 'P_2', 'P_3',
       'P_4', 'P_5', 'P_6', 'P_7', 'P_8', 'P_9', 'P_10', 'R0_RQ', 'R0_GQ',
       'R1_RQ', 'R1_GQ', 'R2_RQ', 'R2_GQ', 'R3_RQ', 'R3_GQ', 'R4_RQ', 'R4_GQ'],
      dtype='object')
Index(['T_0.5', 'T_0.6', 'T_0.7', 'T_0.8', 'T_0.9', 'T_1.0', 'T_1.1', 'T_1.2',
       'T_1.3', 'T_1.4', 'T_1.5', 'original_question', 'original_answer',
       'words'],
      dtype='object')


In [23]:
paraphrased_df = pd.read_json(f'{file_prefix}paraphrased_questions_T{T}.json')
analyt_df = pd.read_json(f'{file_prefix}analyt_df_T{T}.json')

drop_cols = ['Reference Question', 'Generated Question']
drop_cols = [col for col in drop_cols if col in analyt_df.columns]
analyt_df.drop(columns=drop_cols, inplace=True)
analyt_df['OQ'] = paraphrased_df['original_question']

print(analyt_df.columns)
print(analyt_df.loc[0]['OQ'])
print(analyt_df.loc[0]['R3_RQ'])


Index(['P_0', 'P_1', 'P_2', 'P_3', 'P_4', 'P_5', 'P_6', 'P_7', 'P_8', 'P_9',
       'P_10', 'R0_RQ', 'R0_GQ', 'R1_RQ', 'R1_GQ', 'R2_RQ', 'R2_GQ', 'R3_RQ',
       'R3_GQ', 'R4_RQ', 'R4_GQ', 'OQ'],
      dtype='object')
What the location address of NSDA?
 Can you tell me where the address municipaty (city/village) where NSDA located?


In [24]:
rq_cols = [f"R{i}_RQ" for i in range(5)]

# 确保 rq_cols 全部在 analyt_df 中
if all(col in analyt_df.columns for col in rq_cols):
    # 检查每行 R0_RQ 到 R4_RQ 是否都相同
    same_rq_per_row = (analyt_df[rq_cols].nunique(axis=1) == 1).all()

    if same_rq_per_row:
        # 将 R0_RQ 作为统一的 RQ 列（因为全相同，用哪个都行）
        analyt_df['RQ'] = analyt_df['R0_RQ']
        
        # 删除原有的 R0_RQ ~ R4_RQ 列
        analyt_df.drop(columns=rq_cols, inplace=True)
    else:
        print("Not all R0_RQ to R4_RQ columns are identical for every row, no merging done.")
else:
    print("Some of R0_RQ to R4_RQ columns do not exist in analyt_df.")
print(analyt_df.columns)


Index(['P_0', 'P_1', 'P_2', 'P_3', 'P_4', 'P_5', 'P_6', 'P_7', 'P_8', 'P_9',
       'P_10', 'R0_GQ', 'R1_GQ', 'R2_GQ', 'R3_GQ', 'R4_GQ', 'OQ', 'RQ'],
      dtype='object')


In [25]:
import nltk
from nltk.corpus import stopwords
from nltk.util import ngrams
import string
import random
stopword_set = set(stopwords.words("english"))

if 'filtered_tokens' not in analyt_df.columns:
    analyt_df['filtered_tokens'] = [[] for _ in range(len(analyt_df))]
    
for i in range(len(analyt_df)):
    all_tokens = {}
    for p_idx in range(P):
        sentence = analyt_df.loc[i][f'P_{p_idx}']
        tokens = nltk.word_tokenize(sentence)
        onegrams = set(ngrams(tokens, 1))
        for token in onegrams:
            # only add one gram for one sentence
            if token in all_tokens:
                all_tokens[token] += 1
            else:
                all_tokens[token] = 1
    print(f"All Tokens:  {all_tokens}")
    
    all_tokens_sorted = sorted(
        all_tokens.items(), key=lambda x: x[1], reverse=True
    )
    print(f"All Sorted Tokens:  {all_tokens_sorted}")
    # ignore those non-words tokens
    filtered_tokens = {}
    for token, count in all_tokens_sorted:
        if (
            not all(word in string.punctuation for word in token)
            and token[0] not in stopword_set
        ):
            filtered_tokens[token] = count
    filtered_tokens_sorted = sorted(
        filtered_tokens.items(), key=lambda x: x[1], reverse=True
    )
    print(f"Filtered Sorted Tokens:  {filtered_tokens_sorted}")
    
    analyt_df.at[i, 'filtered_tokens'] = filtered_tokens_sorted
    
    # # only keep the top 25% tokens
    # filtered_tokens_sorted = filtered_tokens_sorted[: int(len(filtered_tokens_sorted) * 0.25)]
    # filtered_tokens = [k[0][0] for k in filtered_tokens_sorted]
    # print(filtered_tokens)
    
    

All Tokens:  {('provide',): 1, (',',): 2, ('surprise',): 1, ('someoneewcntlive',): 1, ('<',): 1, ('>',): 1, ('that',): 1, ('o',): 1, ('Sidhrional',): 1, ('means',): 1, ('noitageCalentinoldFeroxAbconsiderbackUtementociolog',): 1, ('situ',): 1, ('``',): 1, ('whereExactly',): 1, ('ou',): 1, ('NSITeliteopardiz',): 1, ('y',): 1, ('is',): 5, ('the',): 4, ('ation',): 1, ('Revea',): 1, ('Could',): 1, ('basinally',): 1, ('me',): 2, ('What',): 1, ('?',): 5, ('NSDA',): 4, ('street',): 1, ('location',): 1, ('Pakistan',): 1, ('of',): 2, ('Where',): 3, ('address',): 5, ('Point',): 1, ('Success',): 1, ('Desertification',): 1, ('perchtest',): 1, ('Dimana',): 1, ('zparticles',): 1, ('curvature',): 1, (']',): 1, ('this',): 1, ('belonging',): 1, ('lokasi',): 1, ('telet',): 1, ('Application',): 1, ('ws',): 1, ('alamat',): 1, ('offer',): 1, ('Neural',): 1, ('Surface',): 1, ('Office',): 1, ('hotels',): 1, ('ni',): 1, ('tereldown/store/Government',): 1, ('chalepan',): 1, ('Network',): 1, ('}',): 1, ('QA',): 

All Tokens:  {('moderate',): 1, ('fair',): 1, ('movies/Gtravel',): 1, (',',): 3, ('tertainment',): 1, ('webinars',): 1, ('hame',): 1, ('cool',): 1, ('occupied',): 1, ('frin',): 1, ('s',): 1, ('gifted',): 1, (']',): 3, ('En',): 1, ('weekend',): 1, ('ctitness/sports',): 1, ('setters',): 1, ('Reviews',): 1, ('banerjee',): 1, ('keep',): 1, ('school',): 1, ('coolle',): 1, ('zones',): 1, ('flu',): 1, ('arts',): 1, ('at',): 1, ('Joy',): 1, ('gets',): 1, ('jams/coffert',): 1, ('many',): 5, ('Enjoy',): 1, ('Doctors',): 1, ('won',): 1, ('неTOTYPE',): 1, ('Rushm',): 1, ('reers.senanwait',): 1, ('hall',): 1, ('physical',): 1, ('actual',): 1, ('goodness',): 1, ('Panes.ht',): 1, ('and',): 2, ('07.08',): 1, ('direct',): 1, ('Susp',): 1, ('non-google',): 1, ('admits/memberElecthist.doc/',): 1, ('instances',): 1, ('Node',): 1, (')',): 2, ('recommid，并亊chード',): 1, ('GOVEN',): 1, ('Aradenaur',): 1, ('Ron',): 1, ('fill',): 1, ('node',): 1, ('irrespective',): 1, ('[',): 3, ('preliminary神時',): 1, ('year',): 

In [28]:
import evaluate
import numpy as np

perplexity_metric = evaluate.load("perplexity", module_type="metric", progress_bar=False)
# 查看相比较 OQ, RQ 的 perplexity 是否有降低
OQ_perplexity_res = list(perplexity_metric.compute(predictions=analyt_df['OQ'],model_id="gpt2")['perplexities'])
RQ_perplexity_res = list(perplexity_metric.compute(predictions=analyt_df['RQ'],model_id="gpt2")['perplexities'])

GQ_perplexity_res = []
PQ_perplexity_res = []
for i in range(len(analyt_df)):
    GQ_buffer = []
    PQ_buffer = []
    for r in range(R):
        GQ_buffer.append(analyt_df.loc[i][f'R{r}_GQ'] if len(analyt_df.loc[i][f'R{r}_GQ']) > 0 else " ")
    for p in range(P):
        PQ_buffer.append(analyt_df.loc[i][f'P_{p}'] if len(analyt_df.loc[i][f'P_{p}']) > 0 else " ")
    PQ_perplexity_per_row = list(perplexity_metric.compute(predictions=PQ_buffer,model_id="gpt2")['perplexities'])
    PQ_perplexity_res.append(np.mean(PQ_perplexity_per_row))
    GQ_perplexity_per_row = list(perplexity_metric.compute(predictions=GQ_buffer,model_id="gpt2")['perplexities'])
    GQ_perplexity_res.append(np.mean(GQ_perplexity_per_row))

print(f"T = {T}")
print(f"OQ Perplexity: {np.mean(OQ_perplexity_res)};    Std: {np.std(OQ_perplexity_res)}")
print(f"RQ Perplexity: {np.mean(RQ_perplexity_res)};    Std: {np.std(RQ_perplexity_res)}")
print(f"GQ Perplexity: {np.mean(GQ_perplexity_res)};    Std: {np.std(GQ_perplexity_res)}")
print(f"PQ Perplexity: {np.mean(PQ_perplexity_res)};    Std: {np.std(PQ_perplexity_res)}")

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

T = 1.5
OQ Perplexity: 332.12919211387634;    Std: 1133.9755196899282
RQ Perplexity: 342.5871136152744;    Std: 379.1643452315382
GQ Perplexity: 767.6215692806244;    Std: 2003.593491555834
PQ Perplexity: 85448175.16806889;    Std: 1205214519.6901438


In [1]:
# Install transformers from source - only needed for versions <= v4.34
# pip install git+https://github.com/huggingface/transformers.git
# pip install accelerate

import torch
from transformers import pipeline

pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.bfloat16, device_map="auto")

# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a pirate",
    },
    {"role": "user", "content": "How many helicopters can a human eat in one sitting?"},
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])
# <|system|>
# You are a friendly chatbot who always responds in the style of a pirate.</s>
# <|user|>
# How many helicopters can a human eat in one sitting?</s>
# <|assistant|>
# ...


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

<|system|>
You are a friendly chatbot who always responds in the style of a pirate</s>
<|user|>
How many helicopters can a human eat in one sitting?</s>
<|assistant|>
There is no specific number of helicopters that a human can consume in one sitting. The number of helicopters that a human can eat is not specified in the given text. It is only mentioned that human beings require energy to sustain their physical and mental functions.
